In [2]:

!git clone https://github.com/MarsImmortal/neural_factorization_machine.git


Cloning into 'neural_factorization_machine'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (68/68), done.
remote: Total 127 (delta 54), reused 83 (delta 34), pack-reused 24 (from 1)
Receiving objects: 100% (127/127), 20.08 MiB | 13.49 MiB/s, done.
Resolving deltas: 100% (59/59), done.


In [3]:
%cd neural_factorization_machine

/content/neural_factorization_machine


In [4]:
import numpy as np
import os

class LoadData(object):
    '''given the path of data, return the data format for DeepFM
    :param path
    return:
    Train_data: a dictionary, 'Y' refers to a list of y values; 'X' refers to a list of features_M dimension vectors with 0 or 1 entries
    Test_data: same as Train_data
    Validation_data: same as Train_data
    '''

    def __init__(self, path, dataset, loss_type):
        self.path = path + dataset + "/"
        self.trainfile = self.path + dataset +".train.libfm"
        self.testfile = self.path + dataset + ".test.libfm"
        self.validationfile = self.path + dataset + ".validation.libfm"
        self.features_M = self.map_features()
        self.Train_data, self.Validation_data, self.Test_data = self.construct_data(loss_type)

    def map_features(self):
        self.features = {}
        self.read_features(self.trainfile)
        self.read_features(self.testfile)
        self.read_features(self.validationfile)
        return len(self.features)

    def read_features(self, file):
        with open(file) as f:
            line = f.readline()
            i = len(self.features)
            while line:
                items = line.strip().split(' ')
                for item in items[1:]:
                    if item not in self.features:
                        self.features[item] = i
                        i += 1
                line = f.readline()

    def construct_data(self, loss_type):
        X_, Y_, Y_for_logloss = self.read_data(self.trainfile)
        Train_data = self.construct_dataset(X_, Y_for_logloss if loss_type == 'log_loss' else Y_)
        X_, Y_, Y_for_logloss = self.read_data(self.validationfile)
        Validation_data = self.construct_dataset(X_, Y_for_logloss if loss_type == 'log_loss' else Y_)
        X_, Y_, Y_for_logloss = self.read_data(self.testfile)
        Test_data = self.construct_dataset(X_, Y_for_logloss if loss_type == 'log_loss' else Y_)
        return Train_data, Validation_data, Test_data

    def read_data(self, file):
        X_ = []
        Y_ = []
        Y_for_logloss = []
        with open(file) as f:
            line = f.readline()
            while line:
                items = line.strip().split(' ')
                Y_.append(float(items[0]))
                v = 1.0 if float(items[0]) > 0 else 0.0
                Y_for_logloss.append(v)
                X_.append([self.features[item] for item in items[1:]])
                line = f.readline()
        return X_, Y_, Y_for_logloss

    def construct_dataset(self, X_, Y_):
        Data_Dic = {}
        X_lens = [len(line) for line in X_]
        indexs = np.argsort(X_lens)
        Data_Dic['Y'] = [Y_[i] for i in indexs]
        Data_Dic['X'] = [X_[i] for i in indexs]
        return Data_Dic

    def truncate_features(self):
        num_variable = len(self.Train_data['X'][0])
        for i in range(len(self.Train_data['X'])):
            num_variable = min(num_variable, len(self.Train_data['X'][i]))
        for i in range(len(self.Train_data['X'])):
            self.Train_data['X'][i] = self.Train_data['X'][i][:num_variable]
        for i in range(len(self.Validation_data['X'])):
            self.Validation_data['X'][i] = self.Validation_data['X'][i][:num_variable]
        for i in range(len(self.Test_data['X'])):
            self.Test_data['X'][i] = self.Test_data['X'][i][:num_variable]
        return num_variable


In [5]:
import tensorflow as tf

class NeuralFM(tf.keras.Model):
    def __init__(self, feature_dim, hidden_factor, layers, keep_prob, loss_type='square_loss', activation='relu'):
        super(NeuralFM, self).__init__()
        self.embedding = tf.keras.layers.Embedding(feature_dim, hidden_factor, mask_zero=False)
        self.hidden_layers = []
        for layer_size in layers:
            self.hidden_layers.append(tf.keras.layers.Dense(layer_size, activation=activation))
        self.output_layer = tf.keras.layers.Dense(1, activation='sigmoid' if loss_type == 'log_loss' else 'linear')
        self.dropout_layers = [tf.keras.layers.Dropout(p) for p in keep_prob]

    def call(self, inputs, training=False):
        x = self.embedding(inputs)
        fm_part = tf.reduce_sum(x, axis=1) ** 2 - tf.reduce_sum(x ** 2, axis=1)
        fm_part = 0.5 * tf.reduce_sum(fm_part, axis=1, keepdims=True)

        x = tf.reduce_sum(x, axis=1)
        for layer, dropout in zip(self.hidden_layers, self.dropout_layers):
            x = layer(x)
            if training:
                x = dropout(x)

        x = tf.concat([x, fm_part], axis=1)
        output = self.output_layer(x)
        return output


In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import Adagrad
from sklearn.metrics import mean_squared_error, log_loss
import numpy as np

# Load data
path = 'data/'
dataset = 'frappe'
loss_type = 'square_loss'  # or 'log_loss'

loader = LoadData(path, dataset, loss_type)
feature_dim = loader.features_M
hidden_factor = 64
layers = [64]
keep_prob = [0.8, 0.5]

# Create model
model = NeuralFM(feature_dim, hidden_factor, layers, keep_prob, loss_type=loss_type, activation='relu')

# Compile model
optimizer = Adagrad(learning_rate=0.05)
model.compile(optimizer=optimizer, loss='binary_crossentropy' if loss_type == 'log_loss' else 'mean_squared_error')

# Prepare data
def prepare_data(data):
    X = tf.keras.preprocessing.sequence.pad_sequences(data['X'], padding='post')
    y = np.array(data['Y'])
    return X, y

X_train, y_train = prepare_data(loader.Train_data)
X_val, y_val = prepare_data(loader.Validation_data)
X_test, y_test = prepare_data(loader.Test_data)

# Train model
history = model.fit(X_train, y_train, epochs=200, batch_size=64, validation_data=(X_val, y_val), verbose=1)

# Evaluate model
y_pred = model.predict(X_test)
if loss_type == 'log_loss':
    score = log_loss(y_test, y_pred)
else:
    score = mean_squared_error(y_test, y_pred)

print(f"Test {loss_type} score: {score}")


Epoch 1/200


/usr/local/lib/python3.10/dist-packages/keras/src/layers/layer.py:372: UserWarning: `build()` was called on layer 'neural_fm', however the layer does not have a `build()` method implemented and it looks like it has unbuilt state. This will cause the layer to be marked as built, despite not being actually built, which may cause failures down the line. Make sure to implement a proper `build()` method.
  warnings.warn(


3157/3157 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.5877 - val_loss: 0.3286
Epoch 2/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.3049 - val_loss: 0.2847
Epoch 3/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.2500 - val_loss: 0.2442
Epoch 4/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 20s 4ms/step - loss: 0.2037 - val_loss: 0.2207
Epoch 5/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.1677 - val_loss: 0.2038
Epoch 6/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 14s 4ms/step - loss: 0.1406 - val_loss: 0.1940
Epoch 7/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.1183 - val_loss: 0.1859
Epoch 8/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 13s 4ms/step - loss: 0.1005 - val_loss: 0.1808
Epoch 9/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 21s 4ms/step - loss: 0.0867 - val_loss: 0.1761
Epoch 10/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 16s 5ms/step - loss: 0.0742 - val_loss: 0.1737
Epoch 11/200
3157/3157 ━━━━━━━━━━━━━━━━━━━━ 14s 5ms/step - loss: 0.0646 - val_loss: 0.1713
Epoch 12/200
3157/31